In [ ]:
!pip install -q --upgrade transformers >=4.41.0 peft==0.10.0 bitsandbytes accelerate pandas==2.2.2


In [ ]:
from peft import prepare_model_for_kbit_training, get_peft_model, LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training


In [ ]:
import pandas as pd
from datasets import Dataset

# Step 1: Load JSONL using pandas
df = pd.read_json("/content/finetune_keywords_dataset_5000.jsonl", lines=True)

# Step 2: Split the text column into prompt and output
df[['prompt', 'output']] = df['text'].str.split('=>', expand=True)
df['prompt'] = df['prompt'].str.strip()
df['output'] = df['output'].str.strip()
df = df.drop(columns=['text'])

# Step 3: Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Step 4: Optional - preview first row
print(dataset[0])


{'prompt': 'list seo keyword ideas for credit repair course florida, texas usa', 'output': 'credit repair course florida, texas usa for marketing agencies, trusted credit repair course florida, texas usa platform usa, automated credit repair course florida, texas usa generator'}


In [ ]:
# Model and dataset paths
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"
DATA_PATH = "/content/finetune_keywords_dataset_5000.jsonl"  # Upload your file here

In [ ]:
import pandas as pd
from datasets import Dataset

# Step 1: Load JSONL using pandas
df = pd.read_json("/content/finetune_keywords_dataset_5000.jsonl", lines=True)

# Step 2: Split the text column into prompt and output
df[['prompt', 'output']] = df['text'].str.split('=>', expand=True)
df['prompt'] = df['prompt'].str.strip()
df['output'] = df['output'].str.strip()
df = df.drop(columns=['text'])

# Step 3: Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Step 4: Optional - preview first row
print(dataset[0])


{'prompt': 'list seo keyword ideas for credit repair course florida, texas usa', 'output': 'credit repair course florida, texas usa for marketing agencies, trusted credit repair course florida, texas usa platform usa, automated credit repair course florida, texas usa generator'}


In [ ]:
from huggingface_hub import login

login("hf_UKTgIm**************lQLNTkJyxan")


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    use_fast=False,
    trust_remote_code=True
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training

MODEL_NAME = "mistralai/Mistral-7B-v0.1"

# Load tokenizer and model in 4-bit mode
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, force_download=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_4bit=True,
    device_map="auto",  # No .to() needed
    trust_remote_code=True
)

# Prepare model for 4-bit training
model = prepare_model_for_kbit_training(model)

# LoRA config
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA
model = get_peft_model(model, lora_config)


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from datasets import Dataset
import json

# Read local JSONL file manually
with open("/content/finetune_keywords_dataset_5000.jsonl", "r") as f:
    lines = [json.loads(line) for line in f]

# Create HuggingFace Dataset object
dataset = Dataset.from_list(lines)

# Split prompt and output
def split_prompt(example):
    parts = example["text"].split("=>")
    return {
        "prompt": parts[0].strip(),
        "output": parts[1].strip()
    }

dataset = dataset.map(split_prompt)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
# Combine prompt and output into a single training text
def format_example(example):
    return {
        "text": f"{example['prompt']}\n{example['output']}"
    }

dataset = dataset.map(format_example)
print(dataset[0])

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

{'text': 'list seo keyword ideas for credit repair course florida, texas usa\ncredit repair course florida, texas usa for marketing agencies, trusted credit repair course florida, texas usa platform usa, automated credit repair course florida, texas usa generator', 'prompt': 'list seo keyword ideas for credit repair course florida, texas usa', 'output': 'credit repair course florida, texas usa for marketing agencies, trusted credit repair course florida, texas usa platform usa, automated credit repair course florida, texas usa generator'}


In [ ]:
# Fix the tokenizer pad token issue
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )

tokenized_dataset = dataset.map(tokenize, batched=True)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/mistral-llm",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1.5,
    fp16=True,  # Use bf16=True if on A100 with support
    logging_steps=10,
    save_steps=200,
    save_total_limit=1,
    learning_rate=2e-4,
    report_to="none"
)


In [ ]:
from transformers import DataCollatorForLanguageModeling

# Define the data collator for causal language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # No masked language modeling for causal models like Mistral
)


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()


<ipython-input-16-04c8215c63be>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, 

Step,Training Loss
10,2.328900
20,1.053500
30,0.708600
40,0.585900
50,0.569000
60,0.550100
70,0.538700
80,0.505900
90,0.428300
100,0.325000


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=938, training_loss=0.3410817201711984, metrics={'train_runtime': 2224.9306, 'train_samples_per_second': 3.371, 'train_steps_per_second': 0.422, 'total_flos': 1.6407410289672192e+17, 'train_loss': 0.3410817201711984, 'epoch': 1.5008})

In [ ]:
save_directory = "/content/drive/MyDrive/mistral-llm"

# Save model and tokenizer
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"Model saved to {save_directory}")

Model saved to /content/drive/MyDrive/mistral-llm


In [ ]:
### load model now ####

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Path to your saved model in Google Drive
model_path = "/content/drive/MyDrive/mistral-llm"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)


# Set the model to evaluation mode
model.eval()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at mistralai/Mistral-7B-v0.1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


MistralForSequenceClassification(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=1024, bias=False)
            (lo

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
model_path = "/content/drive/MyDrive/mistral-llm"

# Load model and tokenizer from Google Drive
model_path = model_path
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
model.eval()

# Input prompt
prompt = "Generate keywords for Online promotion tools"

# Tokenize
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate output
with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=30,
        do_sample=True,
        top_p=0.95,
        temperature=0.7,
        eos_token_id=tokenizer.eos_token_id,
    )

# Decode and print the output (skipping the prompt part)
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
generated_keywords = output_text[len(prompt):].strip()

print("🔑 Generated Keywords:\n", generated_keywords)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🔑 Generated Keywords:
 automated online promotion tools generator, online promotion tools for marketing agencies, online promotion tools for beginners and experts, online promotion tools with ai integration
